Source: https://github.com/krishnaik06/Advanced-House-Price-Prediction-/blob/master/Feature%20Engineering.ipynb

In [42]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
#from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
# Imports preprocessing functions
%run ./v1_jlu_preprocessing.ipynb

## Functions

#### Algorithms

In [2]:
def predictPrice(X_train, X_test, Y_train, Y_test):
    #regr = SVR().fit(X_train, Y_train)
    regr = Ridge(alpha=100).fit(X_train, Y_train)
    
    train_pred = regr.predict(X_train)
    test_pred = regr.predict(X_test)
    
    print("Train MSE: ", mean_squared_error(Y_train, train_pred))
    print("Test MSE: ", mean_squared_error(Y_test, test_pred))

## Code

In [3]:
begin = time.time()
original_df = pd.read_csv('used_cars_data.csv')
end = time.time()

print("Time taken: ", end - begin)

C:\Users\Adeel\AppData\Local\Temp\ipykernel_9280\1749011710.py:2: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  original_df = pd.read_csv('used_cars_data.csv')


Time taken:  78.2944266796112


In [4]:
# Sample a portion of the rows

begin = time.time()
df = original_df.sample(frac=0.30, replace=False, ignore_index=True)
end = time.time()

print("Time taken: ", end - begin)

print(df.shape)
df.head()

Time taken:  1.3972694873809814
(900012, 66)


,vin,back_legroom,bed,bed_height,bed_length,body_type,cabin,city,city_fuel_economy,combine_fuel_economy,...,transmission,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year
0,3FA6P0D98HR303477,38.3 in,NaN,NaN,NaN,Sedan,NaN,Waterbury,20.0,NaN,...,A,6-Speed Automatic,t66972,Platinum AWD,NaN,AWD,All-Wheel Drive,112.2 in,83.5 in,2017
1,1N6ED0EA7LN707211,33.6 in,NaN,--,59.5 in,Pickup Truck,NaN,Stafford,NaN,NaN,...,A,Automatic,t94251,SV Crew Cab RWD,NaN,4X2,4X2,125.9 in,72.8 in,2020
2,2GKFLVE35H6103200,39.9 in,NaN,NaN,NaN,SUV / Crossover,NaN,Hermantown,16.0,NaN,...,A,6-Speed Automatic,t70438,Denali AWD,NaN,AWD,All-Wheel Drive,112.5 in,72.8 in,2017
3,KNMAT2MT5LP535087,37.9 in,NaN,NaN,NaN,SUV / Crossover,NaN,Duarte,26.0,NaN,...,CVT,Continuously Variable Transmission,t86701,SV FWD,NaN,FWD,Front-Wheel Drive,106.5 in,72.4 in,2020
4,KL4MMFS29LB126853,36 in,NaN,NaN,NaN,SUV / Crossover,NaN,Algona,26.0,NaN,...,CVT,Continuously Variable Transmission,t88213,Essence FWD,NaN,FWD,Front-Wheel Drive,102.2 in,71.4 in,2020


#### Initial cleaning

In [5]:
df = cleanColumnsAndRows(df)

(900009, 50)


#### Process columns with years

In [6]:
df["year"] = processYears(df)
df.head()

,vin,back_legroom,body_type,city,city_fuel_economy,daysonmarket,dealer_zip,description,engine_cylinders,engine_displacement,...,torque,transmission,transmission_display,trimId,trim_name,wheel_system,wheel_system_display,wheelbase,width,year
0,3FA6P0D98HR303477,38.3 in,Sedan,Waterbury,20.0,49,6708,Magnetic 2017 Ford Fusion Platinum AWD 6-Speed...,I4,2000.0,...,NaN,A,6-Speed Automatic,t66972,Platinum AWD,AWD,All-Wheel Drive,112.2 in,83.5 in,67
1,1N6ED0EA7LN707211,33.6 in,Pickup Truck,Stafford,NaN,30,77477,We are excited to offer this 2020 Nissan Front...,V6,3800.0,...,"281 lb-ft @ 4,400 RPM",A,Automatic,t94251,SV Crew Cab RWD,4X2,4X2,125.9 in,72.8 in,70
2,2GKFLVE35H6103200,39.9 in,SUV / Crossover,Hermantown,16.0,0,55811,Ebony Twilight Metallic 2017 GMC Terrain Denal...,V6,3600.0,...,"272 lb-ft @ 4,800 RPM",A,6-Speed Automatic,t70438,Denali AWD,AWD,All-Wheel Drive,112.5 in,72.8 in,67
3,KNMAT2MT5LP535087,37.9 in,SUV / Crossover,Duarte,26.0,234,91010,We're Open and We Deliver. Thanks for looking ...,I4,2500.0,...,"175 lb-ft @ 4,400 RPM",CVT,Continuously Variable Transmission,t86701,SV FWD,FWD,Front-Wheel Drive,106.5 in,72.4 in,70
4,KL4MMFS29LB126853,36 in,SUV / Crossover,Algona,26.0,71,50511,Chili Red Metallic 2020 Buick Encore GX Essenc...,I3,1300.0,...,"174 lb-ft @ 1,500 RPM",CVT,Continuously Variable Transmission,t88213,Essence FWD,FWD,Front-Wheel Drive,102.2 in,71.4 in,70


#### Standardize columns of differing data types

In [7]:
df, price_col = convertColumnsToNumericalAndStandardize(df)
df.head()

year


,body_type,city,city_fuel_economy,daysonmarket,engine_cylinders,engine_displacement,engine_type,exterior_color,franchise_dealer,franchise_make,...,model_name,savings_amount,seller_rating,sp_name,transmission,transmission_display,trim_name,wheel_system,wheel_system_display,year
0,0.865101,1.568377,-3.346982e-01,-0.247852,-0.832399,-0.739558,-0.832399,0.060021,-2.064680,1.375811,...,-0.326185,0.668266,0.490179,-1.293004,-0.389739,-1.616526,0.179630,-0.193566,-1.185466,-0.175123
1,-0.704762,1.269130,4.423627e-16,-0.422305,0.688326,0.633308,0.688326,-1.465584,0.484337,0.534219,...,-0.331965,-0.506572,-1.132541,1.274344,-0.389739,0.646011,0.758801,-0.925503,-2.042683,0.545067
2,0.080169,-0.369419,-8.327543e-01,-0.697756,0.688326,0.480767,0.688326,-0.658947,0.484337,-0.695801,...,1.419304,1.301225,0.232228,0.116927,-0.389739,-1.616526,-0.747766,-0.193566,-1.185466,-0.175123
3,0.080169,-0.899447,4.123859e-01,1.450766,-0.832399,-0.358207,-0.832399,0.061583,0.484337,0.534219,...,0.933804,-0.506572,-2.053551,0.619275,0.935280,0.765092,0.759317,0.538371,0.528968,0.545067
4,0.080169,-1.676411,4.123859e-01,-0.045854,-1.035162,-1.273451,-1.035162,-0.989990,0.484337,-1.278442,...,-0.580494,-0.506572,1.429540,0.069643,0.935280,0.765092,-0.565903,0.538371,0.528968,0.545067


#### Run algorithm

In [8]:
# X_train, X_test, Y_train, Y_test = train_test_split(df.to_numpy(), price_col.to_numpy(), test_size=0.2)

# predictPrice(X_train, X_test, Y_train, Y_test)

#LinReg Result

In [9]:
# X_train, X_test, Y_train, Y_test = train_test_split(df.to_numpy(), price_col.to_numpy(), test_size=0.2)

# predictPrice(X_train, X_test, Y_train, Y_test)

# #SVR Result

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(df.to_numpy(), price_col.to_numpy(), test_size=0.2)

In [13]:
param_grid = {'alpha':[1.5,2,2.5,3,3.5,4,10,20,30,40,50,100], 'solver':['auto', 'svd']}

In [24]:
ridgeModel = Ridge()

In [26]:
std_slc = StandardScaler()

In [29]:
pca = PCA()

In [31]:
pipe = Pipeline(steps=[("std_slc", std_slc),
                           ("pca", pca),
                           ("ridge", ridgeModel)])

In [32]:
n_components = list(range(1,X_train.shape[1]+1,1))

In [33]:
normalize = [True, False]

In [34]:
solver = ["auto", "svd", "cholesky", "lsqr", "sparse_cg", "sag", "saga"]

In [35]:
parameters = dict(pca__n_components=n_components,
                      ridge__normalize=normalize,
                      ridge__solver=solver)

In [36]:
clf_GS = GridSearchCV(pipe, parameters)

In [37]:
clf_GS.fit(X_train, Y_train)

C:\Users\Adeel\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Ridge())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  warnings.warn(
C:\Users\Adeel\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklea

C:\Users\Adeel\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Ridge())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  warnings.warn(
C:\Users\Adeel\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklea

C:\Users\Adeel\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Ridge())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  warnings.warn(
C:\Users\Adeel\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklea

C:\Users\Adeel\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(
C:\Users\Adeel\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(
C:\Users\Adeel\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. P

C:\Users\Adeel\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(
C:\Users\Adeel\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(
C:\Users\Adeel\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. P

...

C:\Users\Adeel\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Ridge())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  warnings.warn(
C:\Users\Adeel\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklea

C:\Users\Adeel\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Ridge())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  warnings.warn(
C:\Users\Adeel\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklea

C:\Users\Adeel\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(
C:\Users\Adeel\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(
C:\Users\Adeel\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. P

C:\Users\Adeel\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Ridge())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  warnings.warn(


GridSearchCV(estimator=Pipeline(steps=[('std_slc', StandardScaler()),
                                       ('pca', PCA()), ('ridge', Ridge())]),
             param_grid={'pca__n_components': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
                                               11, 12, 13, 14, 15, 16, 17, 18,
                                               19, 20, 21, 22, 23, 24, 25, 26,
                                               27, 28, 29, 30],
                         'ridge__normalize': [True, False],
                         'ridge__solver': ['auto', 'svd', 'cholesky', 'lsqr',
                                           'sparse_cg', 'sag', 'saga']})

In [39]:
print("Best Number Of Components:", clf_GS.best_estimator_.get_params()["pca__n_components"])
print(); 
print(clf_GS.best_estimator_.get_params()["ridge"])

Best Number Of Components: 28

Ridge(normalize=True, solver='saga')


In [ ]:
CV_Result = cross_val_score(clf_GS, X_train, Y_train, cv=10, n_jobs=-1, scoring="r2")
print(CV_Result)
print(CV_Result.mean())
print(CV_Result.std())

In [22]:
# grid =  GridSearchCV(predictPrice(X_train, X_test, Y_train, Y_test), param_grid)

In [ ]:
# grid =  GridSearchCV(predictPrice(X_train, X_test, Y_train, Y_test), param_grid)

In [21]:
# grid.fit(X_train, Y_train)

In [19]:
#print(grid.best_params_)
# grid_predictions = grid.predict(X_test)

In [17]:
# predictPrice(X_train, X_test, Y_train, Y_test)

Train MSE:  165271456.03523815
Test MSE:  155831427.12200677
